In [ ]:
# Analyzer configuration
dataset_path = '/gdrive/My Drive/vaico-local/datasets/baluarte/analysis-test/'
mode = 'predict'

In [ ]:
# Prediction model paths
model_root = '/gdrive/My Drive/vaico-local/SURA/modelos/' 
workers_path = model_root + 'personas_yolov4-20200801.tflite'
edges_path = model_root + 'bordes_Unet-20200901.tflite'
helmet_path = model_root + 'cascos_resnet50-20200901.ml'
harness_path = model_root + 'arnes_resnet50-20200901.ml' 

In [ ]:
repo_path = '/gdrive/My\ Drive/vaico-local/' +'/Repos/'
mlgeometry = repo_path + 'MLgeometry-0.1.9.tar.gz'
mlevaluator = repo_path + 'MLevaluator-0.0.1.tar.gz'
mlinference = repo_path + 'mlinference-0.0.1.tar.gz'
mldrawer = repo_path + 'MLdrawer-0.0.1.tar.gz'
mlanalyzer = repo_path + 'mlanalyzer-0.0.1.tar.gz'

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
!pip install $mlgeometry $mlinference $mlanalyzer

Processing /gdrive/My Drive/vaico-local/Repos/MLgeometry-0.1.9.tar.gz
Processing /gdrive/My Drive/vaico-local/Repos/mlinference-0.0.1.tar.gz
Processing /gdrive/My Drive/vaico-local/Repos/mlanalyzer-0.0.1.tar.gz
     |████████████████████████████████| 516.2MB 30kB/s 
     |████████████████████████████████| 378kB 29.0MB/s 
     |████████████████████████████████| 3.0MB 37.7MB/s 
     |████████████████████████████████| 460kB 31.2MB/s 
     |████████████████████████████████| 51kB 5.3MB/s 
  Created wheel for MLgeometry: filename=MLgeometry-0.1.9-cp36-none-any.whl size=9701 sha256=30b8929d304c00408d36703441b51ce6754d932428bdbcc61d7650eb5935cd13
  Stored in directory: /root/.cache/pip/wheels/42/d1/da/3f8ae62f7d3936cc0d45dd8c36d7d912e2505002cb30f48991
  Created wheel for mlinference: filename=mlinference-0.0.1-cp36-none-any.whl size=70230 sha256=a632fde89b8732eb7dd64d7237a58c2a31ea1fbe7358795f2b983a64ddbc049a
  Stored in directory: /root/.cache/pip/wheels/47/0d/c6/5ae216606abe04888445fa3e1bf32

In [ ]:
from MLinference.architectures import Yolo4, KerasClassifiers, UNet, OnEdge
from MLinference.strategies import Posterior, Multi, Cascade

Using TensorFlow backend.


In [ ]:
# Create SURA models
worker_detector = Yolo4.load(workers_path, labels={0:'persona'}, input_size=608)
helmet_classifier = KerasClassifiers.load(helmet_path)
harness_classifier = KerasClassifiers.load(harness_path)

worker_cascade = Cascade(
    main_model={'model':worker_detector},
    sub_models={'persona': [
        {'model': helmet_classifier},
        {'model': harness_classifier}
        ]}
)

edges_mask = UNet.load(edges_path, labels={0:'borde'})
on_edge = OnEdge(None, interest_labels=['persona'], mask_label='borde', labels={0:'lejos', 1:'cerca'})
main_models = Multi(models=[worker_cascade, edges_mask])

model = Posterior(models=[main_models, on_edge])

In [ ]:
from MLanalyzer.analyzer import analyzer

In [ ]:
def saving_condition(obj):
    if obj.label == 'borde':
        return False
    return True

In [ ]:
# Analyze dataset
analyzer(dataset_path, model=model, mode=mode, saving_condition=saving_condition )

Running Analyzer. Mode: predict
Making predictions on /gdrive/My Drive/vaico-local/datasets/baluarte/analysis-test/


  0%|          | 0/60 [00:00<?, ?it/s]

 - Predicting images from:/gdrive/My Drive/vaico-local/datasets/baluarte/analysis-test/
 - Saving predictions in in /gdrive/My Drive/vaico-local/datasets/baluarte/analysis-test/predictions.json


/usr/local/lib/python3.6/dist-packages/MLinference/architectures/OnEdge.py:57: RuntimeWarning: Mean of empty slice.
  score = crop[crop>0].mean()
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 60/60 [13:07<00:00, 13.13s/it]

Done
